## Assignment-07 First Step of using machine learning and models.

In [60]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyltp import Segmentor
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import scipy
%matplotlib inline

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [61]:
fname = "D:\\Github\\NLP\\Artificial_Intelligence_for_NLP\\Week_05_0803_pyltp\\sqlResult_1558435.csv"

In [82]:
data = pd.read_csv(fname, encoding='gb18030')

In [83]:
data.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [64]:
xinhua_news = data[data.source == '新华社']

In [65]:
len(xinhua_news) / len(data)

0.8778051801676133

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1566105348906&di=ee9a2de91207767364853d4decc6cca3&imgtype=0&src=http%3A%2F%2Fmmbiz.qpic.cn%2Fmmbiz_png%2FTicO2kbP6Ao5sCsSQDpehZiczLdC6hDCNvoicjcOCEKX2bLxBc9gVOw28zHyFibfIWq9ceRibP6HDTKReGkr6YyTfQQ%2F640%3Fwx_fmt%3Dpng)

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

In [84]:
label = (data.source == '新华社').astype(int)
content = data.content

In [85]:
data_new = pd.DataFrame({'content':content, 'label':label})

In [86]:
data_new.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


### Step3: 使用tfidf进行文本向量化

#### 新闻内容切词整理 

In [87]:
ltp_model_path = "D:\\Github\\NLP\\Projects\\data\\ltp_data_v3.4.0"
segmentor_model_path = os.path.join(ltp_model_path, 'cws.model')
segmentor = Segmentor()
segmentor.load(segmentor_model_path)

In [88]:
def ltp_cut(document):
    if not isinstance(document, str):
        return 'null'
    words = segmentor.segment(document)
    words = [re.findall('\d*\w+', x) for x in words]
    result = ' '.join([x[0] for x in words if x and x[0] !='n'])  # pyltp会把换行符\n给分开
    return result

#### 切词保存数据，这一步还是要花几分钟的

In [90]:
# data_new['content_cut'] = data_new.content.apply(lambda i: ltp_cut(i))
# data_new.to_csv('cleaned_data.csv')

In [143]:
data_new.head()

,content,label,content_cut
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,此外 自 本周 6月 12日 起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已...
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,骁龙 835 作为 唯一 通过 Windows 10 桌面 平台 认证 的 ARM 处理器 ...
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,此前 的 一 加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格...
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,这 是 6月 18日 在 葡萄牙 中部 大佩德罗冈 地区 拍摄 的 被 森林 大火 烧毁 的...
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,原 标题 44 岁 女子 跑 深圳 约会 网友 被 拒 暴雨 中 裸身 奔走 深圳交警 微博...


In [92]:
# index_col 参数不可省略，否则会出现未命名列
data = pd.read_csv('cleaned_data.csv', index_col=0)

In [126]:
data.head()

,content,label,content_cut
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,此外 自 本周 6月 12日 起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已...
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,骁龙 835 作为 唯一 通过 Windows 10 桌面 平台 认证 的 ARM 处理器 ...
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,此前 的 一 加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格...
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,这 是 6月 18日 在 葡萄牙 中部 大佩德罗冈 地区 拍摄 的 被 森林 大火 烧毁 的...
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,原 标题 44 岁 女子 跑 深圳 约会 网友 被 拒 暴雨 中 裸身 奔走 深圳交警 微博...


#### 去除np.nan值

In [158]:
nan_index = [i for i, x in enumerate(data.content) if x is np.nan]
data = data.drop(index=nan_index)

#### 文本向量化

In [163]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=10000)
sample_num = 1000
sub_data = data.content_cut[:sample_num]

In [164]:
X = vectorized.fit_transform(sub_data)

In [165]:
len(vectorized.get_feature_names())

10000

In [166]:
import matplotlib
matplotlib.__version__

'3.1.1'

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. [reference](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

+ 参照scikit learning的示例，从构建你的第一个KNN机器学习模型。

### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

### Step6: 模型测试test_data

### Step7: 调整不同的参数，观察变化

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

### Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。